In [2]:
load(file='data/rtb2_train.rdata')

In [3]:
nfeat=20
rtb3 = rtb2_train[1:(nfeat+1)]
y=as.matrix(rtb3[,1])
xmat = model.matrix(paying_price~., data=rtb3)

In [4]:
dim(xmat)
dim(rtb2_train)
dim(y)

[1] 114825     21

[1] 114825   2838

[1] 114825      1

In [94]:
N = nrow(y)

lambda = 0.001 * N
M = dim(xmat)[2]
I = diag(M)

w = solve(lambda * I + t(xmat) %*% xmat) %*% t(xmat) %*% y
e0 = y - xmat %*% w
beta = drop(N / (t(e0) %*% e0))
print("beta")
beta
alpha = drop(lambda %*% beta)
print("alpha" )
alpha


A = alpha * I + beta * t(xmat) %*% xmat
mN = beta * solve(A) %*% t(xmat) %*% y

lambda = eigen(beta * t(xmat) %*% xmat)$values

# gamma = M * (lambda/ (alpha + lambda))
gamma = 0
for(i in 1:M){
    gamma = gamma + (lambda[i] / (alpha + lambda[i]))
}

print("alpha new" )
alpha_new = drop(gamma / (t(mN) %*% mN))
alpha_new
e1 = y - xmat %*% mN
beta_new = drop((N - gamma) / (t(e1) %*% e1))
print("beta new")
beta_new

while(abs(alpha - alpha_new) + abs(beta - beta_new) >= 10 ** (-5)){
    alpha = alpha_new
    beta = beta_new
    A = alpha * I + beta * t(xmat) %*% xmat
    mN = beta * solve(A) %*% t(xmat) %*% y
    lambda = eigen(beta * t(xmat) %*% xmat)$values
    gamma = 0
    for(i in 1:M){
        gamma = gamma + (lambda[i] / (alpha + lambda[i]))
    }
    alpha_new = drop(gamma / (t(mN) %*% (mN)))
    e1 = y - xmat %*% mN
    beta_new = drop((N - gamma) / (t(e1) %*% e1))
    
}

[1] "beta"


[1] 0.0002536339

[1] "alpha"


[1] 0.02912351

[1] "alpha new"


[1] 0.001006605

[1] "beta new"


[1] 0.0002536024

In [105]:

alpha
beta
mN
diag(sqrt(solve(A)))

[1] 0.0007585334

[1] 0.0002538865

(Intercept),98.941001
agent_1,43.453947
agent_2,114.196607
agent_3,6.545915
agent_4,-15.585473
agent_5,1.861077
agent_6,6.271456
agent_7,17.880470
agent_8,5.386568
agent_9,-12.751414
agent_10,-3.897748


Warning message in sqrt(solve(A)):
“NaNs produced”

(Intercept)     agent_1     agent_2     agent_3     agent_4     agent_5 
  3.0052509   5.6445189   4.3651917   5.3672246   3.7451977   3.2022098 
    agent_6     agent_7     agent_8     agent_9    agent_10    agent_11 
  7.7406664   7.5503850   6.3772277   3.9300660   0.6215702   1.0467525 
   agent_12    agent_13    agent_14    agent_15    agent_16    agent_17 
  2.9594740  24.2552267  24.7024972  11.4676217   5.3772346   3.8749929 
   agent_18    agent_19    agent_20 
  3.6493229   2.4588253   3.0145566

In [109]:
lm_evmax <- function(y, xmat){
    N = nrow(y)

    lambda = 0.001 * N
    M = dim(xmat)[2]
    I = diag(M)

    w = solve(lambda * I + t(xmat) %*% xmat) %*% t(xmat) %*% y
    e0 = y - xmat %*% w
    beta = drop(N / (t(e0) %*% e0))
    alpha = drop(lambda %*% beta)
    A = alpha * I + beta * t(xmat) %*% xmat
    mN = beta * solve(A) %*% t(xmat) %*% y
    lambda = eigen(beta * t(xmat) %*% xmat)$values
    
    # compute new alpha, beta, gamma
    gamma = 0
    for(i in 1:M){
        gamma = gamma + (lambda[i] / (alpha + lambda[i]))
    }
    alpha_new = drop(gamma / (t(mN) %*% mN))
    e1 = y - xmat %*% mN
    beta_new = drop((N - gamma) / (t(e1) %*% e1))

    # iterate
    while(abs(alpha - alpha_new) + abs(beta - beta_new) >= 10 ** (-5)){
        alpha = alpha_new
        beta = beta_new
        A = alpha * I + beta * t(xmat) %*% xmat
        mN = beta * solve(A) %*% t(xmat) %*% y
        lambda = eigen(beta * t(xmat) %*% xmat)$values
        gamma = 0
        for(i in 1:M){
            gamma = gamma + (lambda[i] / (alpha + lambda[i]))
        }
        alpha_new = drop(gamma / (t(mN) %*% (mN)))
        e1 = y - xmat %*% mN
        beta_new = drop((N - gamma) / (t(e1) %*% e1))

    }
    result = list(mN = mN, mNsd = diag(sqrt(solve(A))), alpha = alpha, beta = beta)
    return(result)
}

nfeat=20
rtb3 = rtb2_train[1:(nfeat+1)]
y=as.matrix(rtb3[,1])
xmat = model.matrix(paying_price~., data=rtb3)
lm_evmax(y,xmat )

Warning message in sqrt(solve(A)):
“NaNs produced”

(Intercept),98.941001
agent_1,43.453947
agent_2,114.196607
agent_3,6.545915
agent_4,-15.585473
agent_5,1.861077
agent_6,6.271456
agent_7,17.880470
agent_8,5.386568
agent_9,-12.751414
agent_10,-3.897748


In [2]:
load(file="data/msong_slhw.rdata")
#defin the summary statistic function
sumds = function(ds) {
  nvariable = ncol(ds)
  varnames = colnames(ds)
  for(i in 1:nvariable) {
    tmp1 = ds[,i]
    #remove NA
    tmp1 = tmp1[!is.na(tmp1)]
    n = length(tmp1)
    mean = mean(tmp1)
    median = median(tmp1)
    sd = sd(tmp1)
    q13 = quantile(tmp1, c(0.25, 0.75))
    min1 = min(tmp1)
    max1 = max(tmp1)
    arow = c(n, mean, median, sd, q13, min1, max1)
    if(i==1) {
      out1 = arow
    } else {
      out1 = rbind(out1, arow)
    }
  }
  rownames(out1) = varnames
  colnames(out1) = c("n", "mean", "median", "sd", "Q1", "Q3", "Min", "Max")
  return(out1)
}


In [165]:
x_train = msong_train[1:5000,2:ncol(msong_train)]
y_train = msong_train$year[1:5000]


x_test = msong_test[,2:ncol(msong_test)]
y_test = msong_test$year
dim(x_test)

org_year_mean =  mean(y_train)
# shift to mean
y_train = y_train - mean(y_train)

# x_train <- scale(x_train) fast way to standarize
# standarize x_train
x_train_mean = t(apply(x_train, 2, function(x) mean(x, na.rm=TRUE)))                  
x_train_sd = apply(x_train, 2, function(x) sd(x, na.rm=TRUE))
x_train = sweep(x_train, 2, x_train_mean)
x_train = sweep(x_train, MARGIN=2, x_train_sd,`/`)
                   
                   
#  standarize x_test
x_test = sweep(x_test, 2, x_train_mean)
x_test = sweep(x_test, MARGIN=2, x_train_sd,`/`)                   
y_test = y_test - org_year_mean
                   



[1] 51630    90

In [166]:
length(y_train)

[1] 5000

In [236]:
nfeat = ncol(x_train)

w = rep(0, nfeat)
lr = 0.00001
lambda = 10
w_lr = rep(0, nfeat)

epochs = 300
x_train = as.matrix(x_train)

for(it in 1:epochs){
    w_grad = rep(0, nfeat)
    for(n in 1:nrow(x_train)){
       w_grad =  w_grad - drop( y_train[n]-  w %*% (x_train[n,])) * x_train[n,] + lambda * w
    }
    w = w - lr * w_grad
}

RMSE <- function(x_data, y_data, w){
    RMSE = 0
    SE = 0
    x_data = as.matrix(x_data)
    y_data = as.matrix(y_data)
    print(nrow(y_data))
    for(t in 1:nrow(y_data)){
       SE =  SE + (y_data[t] - (w %*%  x_data[t,])) **2
    }
    RMSE = sqrt(SE / nrow(x_data))
    return(RMSE)
}

RMSE(x_train, y_train, w)
RMSE(x_test, y_test, w)





[1] 5000


10.70827


[1] 51630


10.55976


In [146]:
mean(y_train)
mean(y_test)

[1] 5.166131e-14

[1] 0.3089583

In [108]:
# y_train[1]
w =rep(1, nfeat)
# x_train[1,]

x_train = as.matrix(x_train)


w_grad = rep(0, nfeat)
drop(y_train[1]- w %*% (x_train[1,]))* x_train[1,]



w_grad =  w_grad - ( y_train[1]- w %*% (x_train[1,]))* x_train[1,]

                   
        
                   
                   
                   
                   

x1           x2           x3           x4           x5           x6 
  4.61976921  -0.18410618   7.49173616  -8.95330661  -0.15714067 -10.13728599 
          x7           x8           x9          x10          x11          x12 
-14.49541071 -15.40710208 -10.63319766  -5.60322852   3.27358066   0.46510324 
         x13          x14          x15          x16          x17          x18 
 -0.55717753  -1.46637025 -11.45857317 -13.92378480 -13.50536010 -13.85482083 
         x19          x20          x21          x22          x23          x24 
 -6.82443748  -8.72553764  -7.88233977 -10.32018425 -14.75682838   1.27131226 
         x25          x26          x27          x28          x29          x30 
  1.83819934  -1.56753904   7.23220303  -5.29316128  -5.64937319  -7.22268152 
         x31          x32          x33          x34          x35          x36 
  7.98045527  -7.05557277  -1.98154396  -3.73783841  -0.51048231  -0.32140278 
         x37          x38          x39          x40          x41          x42 
 -1.76231473 -11.84284194   7.84754729  -1.87575649  -9.15359086  -4.30037067 
         x43          x44          x45          x46          x47          x48 
 14.44083194 -10.74534956   6.67555226   1.69251392   1.27794615   1.98516272 
         x49          x50          x51          x52          x53          x54 
  7.59546796  -5.34476633  -7.08893967   2.25196550  -2.11732711   3.09856221 
         x55          x56          x57          x58          x59          x60 
 -3.37887228  -4.79263474   1.18347624  -1.13547524   2.64210755   0.75933369 
         x61          x62          x63          x64          x65          x66 
 -2.56503476   9.27869080   2.82364023  22.07370607   0.01416372  -1.31510543 
         x67          x68          x69          x70          x71          x72 
 -0.78478009   0.30111775   9.62182483   5.72288734  10.47162467  -5.88196350 
         x73          x74          x75          x76          x77          x78 
  1.64450996  -1.69776114  -9.60224628   0.66362510   5.31320095  -1.81087835 
         x79          x80          x81          x82          x83          x84 
  1.34649362   1.85035527 -11.71561681  14.45816091  -3.29577303 -10.19139747 
         x85          x86          x87          x88          x89          x90 
  0.47169434 -11.95685507 -11.61687406  -9.72126678  14.57167946  -2.99370417

Warning message in (y_train[1] - w %*% (x_train[1, ])) * x_train[1, ]:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”